<a href="https://colab.research.google.com/github/wonjae124/2021_Lab_project/blob/main/pytorch_widedeep_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pytorch-widedeep

     |████████████████████████████████| 21.1 MB 1.2 MB/s 
     |████████████████████████████████| 1.4 MB 17.4 MB/s 
     |████████████████████████████████| 52.7 MB 67 kB/s 
     |████████████████████████████████| 397 kB 51.7 MB/s 
     |████████████████████████████████| 134 kB 55.0 MB/s 
     |████████████████████████████████| 1.6 MB 44.3 MB/s 
     |████████████████████████████████| 175 kB 56.3 MB/s 
     |████████████████████████████████| 4.1 MB 52.8 MB/s 
     |████████████████████████████████| 125 kB 56.1 MB/s 
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.44.0
    Uninstalling grpcio-1.44.0:
      Successfully uninstalled grpcio-1.44.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.


In [2]:
pip install git+https://github.com/jrzaurin/pytorch-widedeep.git


  Cloning https://github.com/jrzaurin/pytorch-widedeep.git to /tmp/pip-req-build-lyry6ctp
  Running command git clone -q https://github.com/jrzaurin/pytorch-widedeep.git /tmp/pip-req-build-lyry6ctp


In [5]:
!git clone https://github.com/jrzaurin/pytorch-widedeep
!cd pytorch-widedeep

Cloning into 'pytorch-widedeep'...
remote: Enumerating objects: 5239, done.
remote: Counting objects: 100% (3136/3136), done.
remote: Compressing objects: 100% (1936/1936), done.
remote: Total 5239 (delta 2350), reused 1878 (delta 1121), pack-reused 2103
Receiving objects: 100% (5239/5239), 86.17 MiB | 29.30 MiB/s, done.
Resolving deltas: 100% (3737/3737), done.


In [9]:
!pip install -e .


ERROR: File "setup.py" or "setup.cfg" not found. Directory cannot be installed in editable mode: /content


In [10]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split

from pytorch_widedeep import Trainer
from pytorch_widedeep.preprocessing import WidePreprocessor, TabPreprocessor
from pytorch_widedeep.models import Wide, TabMlp, WideDeep
from pytorch_widedeep.metrics import Accuracy
from pytorch_widedeep.datasets import load_adult


In [12]:
df = load_adult(as_frame=True)
df["income_label"] = (df["income"].apply(lambda x: ">50k" in x)).astype(int)

In [17]:
df.drop("income", axis = 1, inplace= True)
df_train, df_test = train_test_split(df, test_size = 0.2, stratify = df.income_label)


In [25]:
wide_cols = [
             "education",
             "relationship",
             "workclass",
             "occupation",
             "native-country",
             "gender"
]

In [42]:
crossed_cols = [("education","occupation"),("native-country","occupation")] #???

cat_embed_cols = [
                  "workclass",
                  "education",
                  "marital-status",
                  "occupation",
                  "relationship",
                  "race",
                  "gender",
                  "capital-gain",
                  "capital-loss",
                  "native-country",
]  #???

continuous_cols = ["age","hours-per-week"] ##??

target = "income_label" 
target = df_train[target].values

In [43]:
wide_preprocessor = WidePreprocessor(wide_cols = wide_cols, crossed_cols = crossed_cols) #????
X_wide = wide_preprocessor.fit_transform(df_train) # 왜이렇게 많이 뭔가 전처리 하지?

tab_preprocessor = TabPreprocessor(
    cat_embed_cols = cat_embed_cols, continuous_cols= continuous_cols
)

X_tab = tab_preprocessor.fit_transform(df_train)

In [44]:
wide = Wide(input_dim=np.unique(X_wide).shape[0], pred_dim = 1)
tab_mlp = TabMlp(
    column_idx = tab_preprocessor.column_idx,
    cat_embed_input = tab_preprocessor.cat_embed_input,
    continuous_cols = continuous_cols,
)

model = WideDeep(wide = wide, deeptabular = tab_mlp)

In [45]:
trainer = Trainer(model, objective = "binary", metrics = [Accuracy])
# 앗! 여기서 obejctive 말고 다른 것을 할 수 있겠다.
# 그런데, 검증 함수 그런건 어딨냐?.. 아까 있었는데

trainer.fit(
    X_wide=X_wide,
    X_tab = X_tab,
    target = target,
    n_epochs = 5,
    batch_size = 256,
)

epoch 5: 100%|██████████| 153/153 [00:04<00:00, 32.78it/s, loss=2.33e-6, metrics={'acc': 1.0}]


In [47]:
X_wide_te = wide_preprocessor.transform(df_test) #wide랑 tab이랑 무슨 차이지?
X_tab_te = tab_preprocessor.transform(df_test)
preds = trainer.predict(X_wide = X_wide_te, X_tab = X_tab_te) #한쪽은 정답이고 한쪽은 입력 아닌가?

predict: 100%|██████████| 39/39 [00:00<00:00, 73.83it/s]


In [51]:
#trainer와 torch는 무슨 차이지?
trainer.save(path="model_weights", save_state_dict = True)

torch.save(model.state_dict(), "model_weights/wd_model.pt") # 이게 도대체 뭐야? 허허... 왜 파일 확장자가 pt인거지?

model_new = WideDeep(wide = wide, deeptabular = tab_mlp) # wide랑 tab_mlp랑 무슨 차이냐?
model_new.load_state_dict(torch.load("model_weights/wd_model.pt"))

trainer_new = Trainer(model_new, objective = 'binary')

preds = trainer_new.predict(X_wide=X_wide, X_tab=X_tab)

predict: 100%|██████████| 153/153 [00:02<00:00, 53.32it/s]
